In [1]:
from bs4 import BeautifulSoup
import re
import requests

import pandas as pd 

import spacy
import spacy.cli

In [2]:
from time import sleep
from random import randint

In [11]:
dic_sentimentos = {}
lista_sent = ['alegria', 'tristeza', 'raiva', 'nojo', 'medo']
d = 3

def cria_dicionario(sentimento, sent_origem, dist):
    
    if dist > d:
        return
    
    if sentimento in dic_sentimentos:
        return
    
    dic_sentimentos[sentimento] = sent_origem
    
    url = "https://www.sinonimos.com.br/" + sentimento
    soup = BeautifulSoup(requests.get(url).text, "html5lib")
    
    lista_sinonimos = soup.find_all('a', class_ ='sinonimo')

    for item in lista_sinonimos:                
        cria_dicionario(str(item).split('>')[1][:-3], sent_origem, dist+1)
        
    """
    if sentimento in lista_sent:
        for item in lista_sinonimos:
            dic_sentimentos[(str(item).split('>')[1][:-3])] = sentimento
            cria_dicionario(str(item).split('>')[1][:-3], sentimento)
    else:
        for item in lista_sinonimos:                
            dic_sentimentos[(str(item).split('>')[1][:-3])] = sent_origem
    """
        
    #sleep(randint(1,3))

In [4]:
for sentimento in lista_sent:    
    cria_dicionario(sentimento, sentimento, 0)

In [12]:
cria_dicionario('nojo', 'nojo', 0)

In [6]:
(pd.DataFrame.from_dict(dic_sentimentos, orient='index')).to_csv('dic_sentimento.txt', sep='\t' )

In [10]:
pd.DataFrame.from_dict(dic_sentimentos, orient='index', columns = ['sent'])['sent'].value_counts()

alegria     583
raiva       524
tristeza    466
Name: sent, dtype: int64